[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Buddha-subedi/Microwave_Precipitation_Retrievals_from_B-RAINS/blob/main/B-RAINS_demo.ipynb)

# <center> <font color='blue'> **Boosted Transfer Learning for Passive Microwave Precipitation Retrievals** <center>

#### <center> <font color='balck'> Buddha Subedi, Mahyar Garshasbi, Ardeshir Ebtehaj <center>
#### <center> Saint Anthony Falls Laboratory, Department of Civil Environmental and Geo- Engineering, 
#### <center> University of Minnesota <center>
#### <center> Date: May, 2025 (rev 01)

---
<p align="justify"> This notebook representes the results of an algorithm, called the Boosted tRansfer-leArning for PMW precIpitationN Retrievals (B-RAINS) for passive microwave retrieval of precipitation. B-RAINS enables (i) the integration of the information content from ESMs into the retrieval process and (ii) the fusion of multi-satellite observations across varying spatial and temporal resolutions through meta-model learning. The architecture leverages transfer learning by incorporating low-level priors from ESMs and refining them with high-level features derived from satellite-based active retrievals, enhancing accuracy in regions and periods with limited training data. The algorithm first detects the precipitation occurrence and phase and then estimates its rate, while conditioning the results to some key cloud microphysical and environmental variables as further explained later on in Section 1.2. Through ensemble meta-modeling, B-RAINS combines data from multiple spaceborne radars operating at Ka, Ku, and W bands to improve the detection of rainfall and snowfall. Application and validation using data from the Global Precipitation Measurement (GPM) mission and CloudSat's Cloud Profiling Radar (CPR) show that B-RAINS achieves comparable accuracy to existing methods for rain retrievals and delivers substantial improvements in global snowfall estimates when benchmarked against version V07 of the official GPM data products and Multi-Radar/Multi-Sensor System (MRMS) precipitation estimates over the contiguous United States.

This notebook is organized as follows. Section 1, describes the data including the GPM and CloudSat products as well as the ERA5 reanalysis data from the European Centre for Medium-Range Weather Forecasts (ECMWF). Section 2 describes the methodolgy for training the networks, and the last section provides an example for precipitation orbital retrieval using the proposed algorithm.

# **1. Data**

**1.1** **Data Description**

In this study, we considered coincidences of GMI and CPR from April 2014 to July 2019, and coincidences of GMI and DPR data in 2015 and 2023. Furthermore, ancillary information related to atmospheric conditions, surface types and topography are added to all coincidences using the [ERA5](https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5) reanalysis products -- available at spatial resolution of 31km and [GTOPO](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-digital-elevation-global-30-arc-second-elevation-gtopo30) -- available at spatial resolution of 1km. 

**Atmospheric Variables**
*   tclw = Total Column Cloud Liquid Water ($\rm kg/m^2$)
*   tciw = Total Column Cloud Ice Water ($\rm kg/m^2$)
*   tcwv = Total Column Water Vapor ($\rm kg/m^2$)
*   cape = Convective Available Potential Energy ($\rm j/kg$)
*   t2m = 2m air temperature ($\rm K$)
*   u10 = 10m u-component of wind ($\rm m/s$)
*   v10 = 10m v-component of wind ($\rm m/s$)
*   tcslw = Total Column Supercooled Liquid Water ($\rm \frac{kg}{m^2}$)
*   tcw = Total Column Water ($\rm \frac{kg}{m^3}$)
*   cin = Convective Inhibition ($\rm j/kg$)

**Surface Type Variables**
*   lsm = LandSeaMask
*   siconc = Sea Ice Area Fraction
*   sd = Snow Depth ($\rm m$)
*   asn = Snow Albedo (-)
*   rsn = Snow Density
*   swvl1 = Volumetric Soil Water Layer1 (-)

**Orographic Variables**
* Elevation ($\rm m$)
* Aspect

The GMI-CPR coincidences rely on the level-II CloudSat products (R05) including the 2C-PRECIP-COLUMN  and the 2C-SNOW-PROFILE that contains near-surface rain and snowfall rates.  To avoid ground-clutter contamination over complex elevated terrains, the near-surface snowfall rate is reported at 3rd (5th) radar bin above the oceans (land) at 720 (1200) m above the surface.

The 2A-DPR product, retrieved from Ku-band reflectivity, consists the near-surface precipitation phase and its intensity. It should be noted that the DPR algorithm uses precipitation phase at the lowest radar range gate uncontaminated by surface clutter, which may be 0.5--2.0 km above the surface (even over oceans). 

Figure 1, represents the way the coincidences between GMI and CloudSat profiling radar (CPR) and GMI and DPR are collected. For more details with respect to the CloudSat and GMI please refer to ([2](https://journals.ametsoc.org/view/journals/hydr/22/7/JHM-D-20-0296.1.xml))

<div align="center">
  <img src="images/coincidence_image.png" alt="Coincidence Image" width="800"/><br/>
  <em><strong>Figure 1.</strong> Schematic of a CloudSat-GPM coincidence segment (left), and DPR-GMI coincidence (right).</em>
</div>

Resolution mismatches between input and output data sources can introduce information redundancy and exacerbate the inherent non-uniqueness of the inverse problem in precipitation retrievals. In this study, all retrievals are performed at the nominal footprint resolution of GMI pixels (5.1$\times$13.2~km). The total number of samples, after harmonization, in phase detection (rate estimation) is 7,000,000 from ERA5 (2,500,000 rain, 1,000,000 snow), 240,800 from CPR (47,123 snow), and 2,800,000 from DPR (1,700,000 rain) for DPR coincidences. The data sets are partitioned into 70, 15, and 15 percent for training, testing, and validation.

## **1.2 Database Organization and Loading**

**Inputs for the phase detection and rate estimation are as follows:**
> **Inputs:**

*   $\text{TB} = X_{train}(:,1:13)$ (Brightness Temperature at 13 GMI channels [K])
*   $ \text{tciw} = X_{train}(:,14)$ (Cloud Liquid Water Path)
*   $\text{tclw} = X_{train}(:,15)$ (Cloud Ice Water Path)
*   $\text{tcwv} = X_{train}(:,16)$ (2-meter Air Temperature)
*   $\text{t2m} = X_{train}(:,17)$ (Water Vapor Path)
*   $\text{cape} = X_{train}(:,18)$ (Convective Potential Energy)

*   $\text{u10} = X_{train}(:,19)$ (10m u-component of wind)
*   $ \text{v10} = X_{train}(:,20)$ (10m v-component of wind)
*   $\text{skt} = X_{train}(:,21)$ (Skin Temperature)
*   $\text{asn} = X_{train}(:,22)$ (Snow Albedo)
*   $\text{rsn} = X_{train}(:,23)$ (Snow Density)
*   $\text{cin} = X_{train}(:,24)$ (Convective Inhibition)

*   $\text{sd} = X_{train}(:,25)$ (Snow Depth)
*   $ \text{tcslw} = X_{train}(:,26)$ (Total Column Supercooled Liquid Water)
*   $\text{tcw} = X_{train}(:,27)$ (Total Column Water)
*   $\text{swvl1} = X_{train}(:,28)$ (Snow Albedo)
*   $\text{lsm} = X_{train}(:,29)$ (Snow Density)
*   $\text{siconc} = X_{train}(:,30)$ (Convective Inhibition)

*   $\text{Latitude} = X_{train}(:,31)$ (Snow Depth)
*   $ \text{Longitude} = X_{train}(:,32)$ (Total Column Supercooled Liquid Water)
*   $\text{Month} = X_{train}(:,33)$ (Total Column Water)
*   $\text{Day} = X_{train}(:,34)$ (Snow Albedo)
*   $\text{Elevation} = X_{train}(:,35)$ (Snow Density)
*   $\text{Aspect} = X_{train}(:,36)$ (Convective Inhibition)


> **Outputs:**

*   Output of the phase network is a vector containing three integer values **0 (no precipitation)**, **1 (rainfall)**, and **2 (snowfall)**

**For the precipitation estimation, the inputs and outputs are as follows:**

> **Inputs:**

*    The same set of input features that was used for classification is used here as well

> **Outputs**:

*   Output of the estimation network is a vector containing rainfall/snowfall rate [$\rm mm.hr^{-1}$].

## **1.3 Code**
To run this notebook, you need to download this folder and run the code in the Local

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import xgboost as xgb
import os
import scipy.io
import pmw_utils
import importlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, mean_squared_error
import scipy.stats as stats
from scipy.interpolate import interp1d
importlib.reload(pmw_utils)
from pmw_utils import plot_confusion_matrix, BRAINS_model

In [5]:
paths = {
    'training samples':   Path.cwd() / 'data' / 'cpr_phase_train.npz',
    'testing samples':    Path.cwd() / 'data' / 'cpr_phase_test.npz'
}

data = {k: np.load(p) for k, p in paths.items()}
df_train_cpr, df_test_cpr = (pd.DataFrame({k: v[k] for k in v.files}) for v in data.values())

for name, df, sep in zip(
    ['CPR Train sets for Classification', 'CPR Test sets for classification', 'CPR Validation sets for classification'],
    [df_70_train_cpr, df_15_test_cpr, df_15_val_cpr],
    ['##############################', '##############################', '']
):
    counts = df['Prcp flag'].value_counts().sort_index()
    print(f"**{name}**\nTotal samples for classification: {len(df)}")
    print(f"Clear: {counts.get(0,0)}, Rain: {counts.get(1,0)}, Snow: {counts.get(2,0)}")
    if sep: print(sep)

CPR Phase:
  Train shape: (19264, 37)
  Test shape:  (4816, 37)

DPR Phase:
  Train shape: (44800, 37)
  Test shape:  (11200, 37)

ERA5 Phase:
  Train shape: (56000, 37)
  Test shape:  (14000, 37)


In [6]:
from pathlib import Path

# Define the output directory
output_dir = Path(r'G:\Shared drives\SAFL Ebtehaj Group\Buddha Research\Research 1\B-RAINS_demo\data')

# Ensure the output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

# Save function
def save_df_to_npz(df, filename):
    np.savez(output_dir / filename, **{col: df[col].values for col in df.columns})

# Save all DataFrames
save_df_to_npz(df_cpr_phase_train, 'df_cpr_phase_train.npz')
save_df_to_npz(df_cpr_phase_test,  'df_cpr_phase_test.npz')

save_df_to_npz(df_dpr_phase_train, 'df_dpr_phase_train.npz')
save_df_to_npz(df_dpr_phase_test,  'df_dpr_phase_test.npz')

save_df_to_npz(df_era5_phase_train, 'df_era5_phase_train.npz')
save_df_to_npz(df_era5_phase_test,  'df_era5_phase_test.npz')